# NAIVE BAYES CLASSIFICATION
---

In [ ]:
predict_bootstrap <- function(genotype.matrix, clust, bootstrap.size, bootstrap.n, laplace=0.5, 
                              add.gtex=0, predict.del=FALSE, pseudocount=5){                    
    pred.mat <- t(genotype.matrix)
    #remove if missing >10% of data
    missing <- apply(pred.mat, 2, FUN=function(x) sum(is.na(x)))
    pred.mat <- pred.mat[, missing/nrow(pred.mat) <= 0.1]
    samples <- rownames(pred.mat)
    pred.mat <- as.matrix(kNN(pred.mat, k=5, imp_var=FALSE))
    rownames(pred.mat) <- samples

    bootstrap <- function(boot.index){
        bs <- bootstrap_diplotypes(bootstrap.size, clust, pseudocount=pseudocount)
        train.mat <- t(bs$matrix)
        class.lab <- bs$labels
        
        train.mat <- train.mat[,colnames(train.mat) %in% colnames(pred.mat)]
        pred.mat <- pred.mat[,colnames(pred.mat) %in% colnames(train.mat)]

        if (add.gtex > 0){
            duplicated <- rownames(gtex.matrix)[duplicated(rownames(gtex.matrix))]
            train.mat <- train.mat[,colnames(train.mat) %in% rownames(gtex.matrix) &
                                  ! colnames(train.mat) %in% duplicated]
            pred.mat <- pred.mat[,colnames(pred.mat) %in% rownames(gtex.matrix) &
                                  ! colnames(pred.mat) %in% duplicated]
            subset.mat <- gtex.matrix[rownames(gtex.matrix) %in% colnames(train.mat) &
                                  ! rownames(gtex.matrix) %in% duplicated,
                                    gtex.haps$cluster != "."]
            subset.labels <- gtex.haps$cluster[gtex.haps$cluster != "."]

            keep <- sample(1:ncol(subset.mat), add.gtex, replace=FALSE)

            train.mat <- rbind(train.mat, t(subset.mat[,keep]))
            class.lab <- factor(c(as.character(class.lab),
                                  as.character(subset.labels[keep])))
        }
        if(predict.del){
            class.lab <- factor(str_count(class.lab, pattern="D"))
        }

        pnb <- poisson_naive_bayes(x=train.mat, y=class.lab, laplace=laplace)
        #summary(pnb)
        
        #print(pred.mat)
        hap.prediction <- predict(pnb, pred.mat, type="class")
        hap.posterior <- rowMaxs(predict(pnb, pred.mat, type = "prob"))
        
        best_guess <- function(x, labs){ 
            idx = order(x, decreasing=TRUE)[c(1,2)]
            return(data.frame(pred=as.character(labs[idx[1]]), prob=x[idx[1]], 
                              prob2=x[idx[2]], pred2=as.character(labs[idx[2]])))
        }
        
        probabilities <- predict(pnb, pred.mat, type = "prob")
        df <- do.call(rbind, lapply(1:nrow(probabilities), FUN=function(i)
                        best_guess(probabilities[i,], labs=as.character(colnames(probabilities)))))
        df$sample <- samples
        df$boot <- boot.index
        rownames(df) <- c()
        #df <- data.frame(sample=samples, cluster=hap.prediction, posterior=hap.posterior)

        return(df)
    }

    boot.df <- do.call(rbind, lapply(1:bootstrap.n, bootstrap))
    vote <- function(samp){
        tt <- table(boot.df[boot.df$sample == samp,]$pred)
        return(names(tt[which.max(tt)]))   
    }

    haplotype.vote <- sapply(samples, FUN=vote)
    
    predict.df <- data.frame(sample=as.character(samples), 
                     cluster=as.character(haplotype.vote))

    ll <- list(prediction=predict.df, evidence=boot.df)
    return (ll)
}

In [ ]:
set.seed(428)
b.size <- ncol(hap.matrix)*4
b.n <- 11
use.gtex=100
input <- array.matrix
#input <- imputed.matrix
laplace=1
get_platform_matrix <- function(platform, input.matrix){
    samples <- array.df[array.df$platform %in% platform,]$sample
    mat <- as.matrix(input.matrix[,colnames(input.matrix) %in% samples])
    return(mat)
}

platforms <- list(c("Omni5"), c("660W", "660K", "GWAS3"), c("GWAS1"), c("370K"))
mats <- lapply(platforms, FUN=get_platform_matrix, input.matrix=input)

results <- lapply(mats, predict_bootstrap, clust=clusters,
                 bootstrap.size=b.size, bootstrap.n=b.n, add.gtex=use.gtex, laplace=laplace)
predict.df <- do.call(rbind, lapply(results, function(x) x$prediction))
evidence.df <- do.call(rbind, lapply(results, function(x) x$evidence))

results <- lapply(mats, predict_bootstrap, clust=clusters,
                 bootstrap.size=b.size, bootstrap.n=b.n,
                  predict.del=TRUE, add.gtex=use.gtex, laplace=laplace)

predict.del.df <- do.call(rbind, lapply(results, function(x) x$prediction))
evidence.del.df <- do.call(rbind, lapply(results, function(x) x$evidence))

pdf <- data.frame(sample=predict.del.df$sample, 
                  del.prediction=as.numeric(as.character(predict.del.df$cluster)))

predict.df <- merge(predict.df, pdf, by="sample", all.x=TRUE)
predict.df <- merge(predict.df, array.df, by="sample", all.x=TRUE)
                                         
prob.df <- aggregate(evidence.df[,c("prob","prob2")], by=list(sample=evidence.df$sample), FUN=mean)
colnames(prob.df) <- c("sample", "mean.prob", "mean.prob2") 
predict.df <- merge(predict.df, prob.df, by="sample")

prob.del.df <- aggregate(evidence.del.df[,c("prob","prob2")], by=list(sample=evidence.del.df$sample), FUN=mean)
colnames(prob.del.df) <- c("sample", "mean.del.prob", "mean.del.prob2") 
predict.df <- merge(predict.df, prob.del.df, by="sample")

table(predict.df$cluster)

In [ ]:
set.seed(428)
b.size <- ncol(hap.matrix)*4
b.n <- 3
use.gtex=0
input <- gt.matrix

create_platform_matrix <- function(platform){ 
    samples <- array.df[array.df$platform %in% platform,]$sample
    pred.mat <- t(array.matrix[,colnames(array.matrix) %in% samples])
    #remove if missing >10% of data
    missing <- apply(pred.mat, 2, FUN=function(x) sum(is.na(x)))
    pred.mat <- pred.mat[, missing/nrow(pred.mat) <= 0.1]
    pred.mat <- as.matrix(kNN(pred.mat, k=5, imp_var=FALSE))
                     
    return (gt.matrix[rownames(gt.matrix) %in% colnames(pred.mat),])
}

platforms <- list(c("Omni5"), c("660W", "660K", "GWAS3"), c("GWAS1"), c("370K"))
mats <- lapply(platforms, FUN=create_platform_matrix)
                     
results <- lapply(mats, predict_bootstrap, clust=clusters,
                 bootstrap.size=b.size, bootstrap.n=b.n, add.gtex=use.gtex)

names <- c("Omni5", "GWAS3", "GWAS1", "370K")
for (i in 1:length(results)){
    results[[i]]$prediction$platform <- names[i]
    results[[i]]$evidence$platform <- names[i]
}

predict.seq.df <- do.call(rbind, lapply(results, function(x) x$prediction))
evidence.seq.df <- do.call(rbind, lapply(results, function(x) x$evidence))

In [ ]:
plot.df <- merge(evidence.df, array.df, by="sample", all.x=TRUE)
plot.df <- merge(plot.df, predict.df[,c("sample", "cluster")], by="sample", all.x=TRUE)

#for (g in unique(predict.df$cluster)){ hist(predict.df[predict.df$cluster==g,]$count, main=g) }
ggplot(plot.df, aes(x=prob, fill=platform, color=platform)) +
  geom_histogram(alpha=0.5, position="identity", bins=25) + theme_bw() +
  facet_wrap(~platform, scales="free") + ggtitle("Posterior Probabilities")

ggplot(plot.df, aes(x=prob, fill=cluster, color=cluster)) +
  geom_histogram(alpha=0.5, position="identity", bins=25) + theme_bw() +
  facet_wrap(~cluster, scales="free") + ggtitle("Posterior Probabilities")

#confusionMatrix(plot.df$pred, plot.df$pred2)